In [ ]:
// Kotlin notebook - Seven Places Open Stochastic Petri Net
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.api.*

In [ ]:
// Create the model
val model = Network("model")

// Create nodes
val source = Source(model, "Source")
val sink = Sink(model, "Sink")

// Create 7 Places
val P1 = Place(model, "P1")
val P2 = Place(model, "P2")
val P3 = Place(model, "P3")
val P4 = Place(model, "P4")
val P5 = Place(model, "P5")
val P6 = Place(model, "P6")
val P7 = Place(model, "P7")

// Create 8 Transitions
val T1 = Transition(model, "T1")
val T2 = Transition(model, "T2")
val T3 = Transition(model, "T3")
val T4 = Transition(model, "T4")
val T5 = Transition(model, "T5")
val T6 = Transition(model, "T6")
val T7 = Transition(model, "T7")
val T8 = Transition(model, "T8")

// Create job class
val jobclass = OpenClass(model, "Class1", 0)
source.setArrival(jobclass, Exp.fitMean(1.0))

In [3]:
// Configure Transition T1
var mode = T1.addMode("Mode1")
T1.setNumberOfServers(mode, Int.MAX_VALUE)
T1.setDistribution(mode, Exp(4.0))
T1.setEnablingConditions(mode, jobclass, P1, 1)
T1.setFiringOutcome(mode, jobclass, P2, 1)

In [4]:
// Configure Transition T2 (immediate)
mode = T2.addMode("Mode2")
T2.setNumberOfServers(mode, Int.MAX_VALUE)
T2.setEnablingConditions(mode, jobclass, P2, 1)
T2.setFiringOutcome(mode, jobclass, P3, 1)
T2.setTimingStrategy(mode, TimingStrategy.IMMEDIATE)
T2.setFiringPriorities(mode, 1)
T2.setFiringWeights(mode, 1.0)

In [5]:
// Configure Transition T3 (immediate)
mode = T3.addMode("Mode3")
T3.setNumberOfServers(mode, Int.MAX_VALUE)
T3.setEnablingConditions(mode, jobclass, P2, 1)
T3.setFiringOutcome(mode, jobclass, P4, 1)
T3.setTimingStrategy(mode, TimingStrategy.IMMEDIATE)
T3.setFiringPriorities(mode, 1)

In [6]:
// Configure Transition T4 (immediate)
mode = T4.addMode("Mode4")
T4.setNumberOfServers(mode, Int.MAX_VALUE)
T4.setEnablingConditions(mode, jobclass, P3, 1)
T4.setEnablingConditions(mode, jobclass, P5, 1)
T4.setFiringOutcome(mode, jobclass, P5, 1)
T4.setFiringOutcome(mode, jobclass, P6, 1)
T4.setTimingStrategy(mode, TimingStrategy.IMMEDIATE)
T4.setFiringPriorities(mode, 1)

In [7]:
// Configure Transition T5 (immediate with inhibiting condition)
mode = T5.addMode("Mode5")
T5.setNumberOfServers(mode, Int.MAX_VALUE)
T5.setEnablingConditions(mode, jobclass, P4, 1)
T5.setEnablingConditions(mode, jobclass, P5, 1)
T5.setFiringOutcome(mode, jobclass, P7, 1)
T5.setInhibitingConditions(mode, jobclass, P6, 1)
T5.setTimingStrategy(mode, TimingStrategy.IMMEDIATE)
T5.setFiringPriorities(mode, 1)

In [8]:
// Configure Transition T6 (timed with Erlang distribution)
mode = T6.addMode("Mode6")
T6.setNumberOfServers(mode, Int.MAX_VALUE)
T6.setDistribution(mode, Erlang(2.0, 2))
T6.setEnablingConditions(mode, jobclass, P6, 1)
T6.setFiringOutcome(mode, jobclass, P1, 1)

In [9]:
// Configure Transition T7 (timed)
mode = T7.addMode("Mode7")
T7.setNumberOfServers(mode, Int.MAX_VALUE)
T7.setDistribution(mode, Exp(2.0))
T7.setEnablingConditions(mode, jobclass, P7, 1)
T7.setFiringOutcome(mode, jobclass, P1, 1)
T7.setFiringOutcome(mode, jobclass, P5, 1)

In [10]:
// Configure Transition T8 (timed)
mode = T8.addMode("Mode8")
T8.setNumberOfServers(mode, Int.MAX_VALUE)
T8.setDistribution(mode, Exp(2.0))
T8.setEnablingConditions(mode, jobclass, P4, 1)
T8.setFiringOutcome(mode, jobclass, sink, 1)

In [11]:
// Set up routing matrix
val R = model.initRoutingMatrix()

// Source routing
R.set(jobclass, jobclass, source, P1, 1.0)

// Place to Transition routing
R.set(jobclass, jobclass, P1, T1, 1.0)
R.set(jobclass, jobclass, P2, T2, 1.0)
R.set(jobclass, jobclass, P2, T3, 1.0)
R.set(jobclass, jobclass, P3, T4, 1.0)
R.set(jobclass, jobclass, P4, T5, 1.0)
R.set(jobclass, jobclass, P4, T8, 1.0)
R.set(jobclass, jobclass, P5, T4, 1.0)
R.set(jobclass, jobclass, P5, T5, 1.0)
R.set(jobclass, jobclass, P6, T5, 1.0)
R.set(jobclass, jobclass, P6, T6, 1.0)
R.set(jobclass, jobclass, P7, T7, 1.0)

// Transition to Place routing
R.set(jobclass, jobclass, T1, P2, 1.0)
R.set(jobclass, jobclass, T2, P3, 1.0)
R.set(jobclass, jobclass, T3, P4, 1.0)
R.set(jobclass, jobclass, T4, P5, 1.0)
R.set(jobclass, jobclass, T4, P6, 1.0)
R.set(jobclass, jobclass, T5, P7, 1.0)
R.set(jobclass, jobclass, T6, P1, 1.0)
R.set(jobclass, jobclass, T7, sink, 1.0)
R.set(jobclass, jobclass, T7, P1, 1.0)
R.set(jobclass, jobclass, T7, P5, 1.0)
R.set(jobclass, jobclass, T8, sink, 1.0)

model.link(R)

In [ ]:
// Set initial state
source.setState(0)
P1.setState(2)
P2.setState(0)
P3.setState(0)
P4.setState(0)
P5.setState(1)
P6.setState(0)
P7.setState(0)

val state = model.state
println("Initial state: $state")

In [ ]:
import jline.solvers.Solver
import jline.util.matrix.Matrix

// Solve with JMT solver
val options = Solver.defaultOptions()
options.keep = false
options.verbose = VerboseLevel.STD
options.cutoff = Matrix.singleton(10.0)
options.seed = 23000

val solver = JMT(model, options)
val avgTable = solver.avgTable
println("Average Performance Metrics:")
avgTable.print()